[INDEX](Kubernetes.ipynb)  [NEXT](4.5.Traefik.ipynb)     [BELOW - LoadBalancer](#LoadBalancer)

<img align="left" src="https://avatars1.githubusercontent.com/u/13629408?s=400&v=4" width=100 />
# 4. Kubernetes Services

# Exposing applications as a Service
<pre style="align:center" >
For this section of the lab we will continue to work with our 3-node cluster comprised of 1 Master and 2 Workers.
</pre>
<img src="images/KubeNodes_1m_2w.svg.png" width="400" />
    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @containous </td></tr></table>

## Services allow to abstract away Pod details
As mentioned earlier exposing our application as a Service allows to access the application at a single virtual ip address and port - without having to know the ip addresses of the individual pods which may change ip address as pods are scaled up/down or as pods or nodes fail or come on line.

It can also allow load-balancing across pods.

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @containous </td></tr></table>

In [1]:
. ./.shell.functions

Loading bash helper functions, new functions added:
-- checkPresent 
-- checkState 
-- cont 
-- untilNotPresent 
-- untilNotState 
-- untilPresent 
-- untilState 


    
## Accessing our application - via a Service NodePort

We can expose our service using different types.

The first type we will consider is called NodePort and has the effect of exposing our Service on the same port on each Kubernetes node - and so with the ip address of that node.

Let's try that with our service:

In [2]:
checkPresent svc k8s-demo-service &&
    kubectl delete svc/k8s-demo-service
    
kubectl expose deploy k8s-demo --type=NodePort --name=k8s-demo-service

k8s-demo-service   NodePort    10.111.29.175   <none>        8080:32102/TCP   5m
service "k8s-demo-service" deleted
service "k8s-demo-service" exposed


In [3]:
kubectl get svc/k8s-demo-service -o wide

EXPOSED_PORT=$(kubectl get --no-headers=true svc/k8s-demo-service | awk '{ FS=":"; $0=$5; FS="/"; $0=$2; print $1;}')
CLUSTER_IP=$(kubectl get --no-headers=true svc/k8s-demo-service | awk '{ print $3;}')

echo; echo "----"
echo EXPOSED_PORT=$EXPOSED_PORT
echo CLUSTER_IP=$CLUSTER_IP

NAME               TYPE       CLUSTER-IP    EXTERNAL-IP   PORT(S)          AGE       SELECTOR
k8s-demo-service   NodePort   10.108.8.95   <none>        8080:31897/TCP   1m        app=k8s-demo

----
EXPOSED_PORT=31897
CLUSTER_IP=10.108.8.95


In [4]:
kubectl scale --replicas=3 deploy/k8s-demo

deployment "k8s-demo" scaled


In [5]:
kubectl get all -o wide

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   3         3         3            3           52m       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-6b76bd84d7   3         3         3         52m       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=2632684083

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   3         3         3            3           52m       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-6b76bd84d7   3         3         3         52m       k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=2632684083

NAME              

## NOTE: No pods are scheduled to the Master node

We would need to have a "tainted" Master node to allow Pods to be scheduled there (Minikube which is a single-node cluster obviously allows this).

Note below that the pods are deployed to Worker nodes kube-node-1 and kube-node-2.

In [6]:
kubectl get pods -o wide

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-6b76bd84d7-4rfz5   1/1       Running   0          52m       10.192.2.3   kube-node-1
k8s-demo-6b76bd84d7-7ljmj   1/1       Running   0          4m        10.192.3.4   kube-node-2
k8s-demo-6b76bd84d7-bvsxh   1/1       Running   0          52m       10.192.3.2   kube-node-2


Let's try to access the Pods directly on the Worker nodes:

In [1]:
NODE1_IP=$(kubectl describe node kube-node-1 | grep InternalIP: | sed 's/.*: *//')
NODE2_IP=$(kubectl describe node kube-node-2 | grep InternalIP: | sed 's/.*: *//')

echo "NODE1_IP=$NODE1_IP"
echo "NODE2_IP=$NODE2_IP"

NODE1_IP=10.192.0.3
NODE2_IP=10.192.0.4


In [8]:
curl 10.111.29.175:32102

In [ ]:
curl 10.111.29.175:8080

In [5]:
curl $NODE2_IP:$EXPOSED_PORT

curl: (7) Failed to connect to 10.192.0.4 port 31897: Connection refused


: 7

In [4]:
curl $NODE1_IP:$EXPOSED_PORT

curl: (7) Failed to connect to 10.192.0.3 port 31897: Connection refused


: 7

In [ ]:
kubectl describe svc/k8s-demo-service

#### For more information about using NodePort see [HERE](https://kubernetes.io/docs/tasks/access-application-cluster/service-access-application-cluster/)

**Note**: Although there is a Cluster IP there is no Service IP assigned, because we are using NodePort

So we can access our Pods by connecting to the hosting node but not to the Service ... so far ...

<a name="LoadBalancer"> </a>

## Accessing our application - via a Service LoadBalancer


In [ ]:
#kubectl delete svc/k8s-demo-service
kubectl delete svc/k8s-demo-service-lb
#kubectl expose deploy k8s-demo --type=LoadBalancer --name=k8s-demo-service-lb

RS_K8SDEMO=$(kubectl get rs | grep k8s-demo | awk '{ print $1;}')

#kubectl expose deploy k8s-demo --type=LoadBalancer --name=k8s-demo-service-lb

kubectl expose rs $RS_K8SDEMO --type="LoadBalancer" --name="k8s-demo-service-lb"

In [ ]:
kubectl get svc -o wide

### LoadBalancer not working with kubeadm/minikube

As described [here](https://forums.manning.com/posts/list/42097.page) currently Kubernetes implementations created with kubeadm (which was used to prepare this cluster) and minikube do not yet implement tLoadBalancer Service.

In [ ]:
#untilNotState svc k8s-demo-service-lb pending

In [ ]:
kubectl describe svc/k8s-demo-service-lb

LB_EXPOSED_PORT=$(kubectl get --no-headers=true svc/k8s-demo-service-lb | awk '{ FS=":"; $0=$5; FS="/"; $0=$2; print $1;}')
LB_CLUSTER_IP=$(kubectl get --no-headers=true svc/k8s-demo-service-lb | awk '{ print $3;}')

echo
echo LB_EXPOSED_PORT=$LB_EXPOSED_PORT
echo LB_CLUSTER_IP=$LB_CLUSTER_IP
echo MASTER_IP=$MASTER_IP

In [ ]:
curl ${LB_CLUSTER_IP}:$LB_EXPOSED_PORT

In [ ]:
curl ${MASTER_IP}:$EXPOSED_PORT

In [ ]:
kubectl describe no/kube-master | grep "InternalIP:"

MASTER_IP=$(kubectl describe no/kube-master | grep "InternalIP:" | sed -e 's/.*: *//')
echo MASTER_IP=$MASTER_IP
WORKER1_IP=$(kubectl describe no/kube-node-1 | grep "InternalIP:" | sed -e 's/.*: *//')
echo WORKER1_IP=$WORKER1_IP
WORKER2_IP=$(kubectl describe no/kube-node-2 | grep "InternalIP:" | sed -e 's/.*: *//')
echo WORKER2_IP=$WORKER2_IP

#curl $MASTER_IP:8080
curl $WORKER2_IP:8080
curl $WORKER2_IP:$EXPOSED_PORT
curl $WORKER1_IP:8080
curl $WORKER1_IP:$EXPOSED_PORT
curl $MASTER_IP:$EXPOSED_PORT

#### For more information about using LoadBalancer see [HERE](https://kubernetes.io/docs/tasks/access-application-cluster/load-balance-access-application-cluster/)


[INDEX](Kubernetes.ipynb)  [NEXT](4.5.Traefik.ipynb)
